In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 7.4 MB/s 
     |████████████████████████████████| 182 kB 67.5 MB/s 
     |████████████████████████████████| 7.6 MB 47.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 6.1 MB/s 


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("Vamsi/T5_Paraphrase_Paws")  
model = AutoModelForSeq2SeqLM.from_pretrained("Vamsi/T5_Paraphrase_Paws").to('cuda')

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
sentence = "This is something which i cannot understand at all"

text =  "paraphrase: " + sentence + " </s>"

encoding = tokenizer.encode_plus(text, max_length=280, padding='max_length', return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to("cuda"), encoding["attention_mask"].to("cuda")

In [ ]:
outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    max_length=280,
    do_sample=True,
    top_k=120,
    top_p=0.95,
    early_stopping=True,
    num_return_sequences=5
)

In [ ]:
for output in outputs:
    line = tokenizer.decode(output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    print(line)

This is something i can only understand.
This is something which i can only understand at first.
This is something that I cannot understand at all.
This is something which i cannot understand at all.
This is something that i can't understand at all.


In [ ]:
import nltk
nltk.download('punkt')
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu, modified_precision

def tokenize(text):
    tokenized_data = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            tokenized_data.append(word)
    return tokenized_data

def get_bleu_score(reference, candidate):
    reference = [tokenize(reference)]
    candidate = tokenize(candidate)
    bleu_score = float(modified_precision(reference, candidate, n=1))
    return bleu_score

def get_rid_of_useless_symbols(text):
    return_text = ''
    for i in range(len(text)):
        if text[i] == '@' or text[i-1] == '@':
            continue
        return_text = return_text + text[i]
    return return_text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# creating (query, target) DataFrame from dataset
import os

from zipfile import ZipFile
from tqdm.notebook import tqdm
# change the file name as per your setup
with ZipFile('/content/drive/MyDrive/CSE 573: Stance Detection /Datasets/SemEval-2016/stance-data-all-annotations.zip') as z:
    z.extractall()
dataDir = './data-all-annotations'
import pandas as pd
with open(os.path.join(dataDir, 'trainingdata-all-annotations.txt'), encoding='windows-1252') as f:
    lines = f.readlines()
queries = []
targets = []
stances = []
maxLen = 0
for line in tqdm(lines[1:]):
    temp = line.split('\t')
    maxLen = max(maxLen, len(temp[2]))
    queries.append(temp[2].lower())
    targets.append(temp[1].lower())
    stances.append(temp[3])

print(queries[0], targets[0], maxLen)
df= pd.DataFrame(zip(queries, targets, stances), columns=['Query', 'Target', 'Stance'])

possible_stance = df.Stance.unique()
stance_dict = {}
for index, possible_stance in enumerate(possible_stance):
    stance_dict[possible_stance] = index
df['Stance'] = df.Stance.replace(stance_dict)

possible_target = df.Target.unique()
target_dict = {}
for index, possible_target in enumerate(possible_target):
    target_dict[possible_target] = index
df['Target'] = df.Target.replace(target_dict)
print(df.Target.unique(), df.Stance.unique(), len(df))

  0%|          | 0/2814 [00:00<?, ?it/s]

dear lord thank u for all of ur blessings forgive my sins lord give me strength and energy for this busy day ahead #blessed #hope #semst atheism 142
[0 1 2 3 4] [0 1 2] 2814


In [ ]:
# Open Procon20

usingProcon = True

if usingProcon:
    proconfile_train = open('/content/drive/My Drive/CSE 573: Stance Detection /Datasets/Procon20/procon_final.csv', encoding='utf8', errors='backslashreplace')
    procon_train = pd.read_csv(proconfile_train, sep=',', header=0)

    # Create a copy and backtranslate the tweets
    df2 = procon_train.copy()

    possible_target2 = df2.Target.unique()
    target_dict2 = {}
    for index, possible_target in enumerate(possible_target2):
        target_dict2[possible_target] = index
    df2['Target'] = df2.Target.replace(target_dict2)
    print(df2.Target.unique(), df2.Stance.unique(), len(df2))

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144] [0 1] 5984


In [ ]:
import numpy as np
from torch.utils.data import TensorDataset
import torch

encoded_data = tokenizer.batch_encode_plus(list(df.Query.values),
                                           max_length=280,
                                           padding='max_length',
                                           return_tensors="pt")

targets = torch.tensor(df.Target.values)
stances = torch.tensor(df.Stance.values)


input_ids = encoded_data['input_ids']
attention_masks = encoded_data['attention_mask']

dataset = TensorDataset(input_ids, attention_masks, targets, stances)

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 16

dataloader = DataLoader(dataset, 
                              sampler=RandomSampler(dataset), 
                              batch_size=batch_size)

In [ ]:

if usingProcon:
  encoded_data2 = tokenizer.batch_encode_plus(list(df2.Query.values),
                                            max_length=3070,
                                            padding='max_length',
                                            return_tensors="pt")

  targets2 = torch.tensor(df2.Target.values)
  stances2 = torch.tensor(df2.Stance.values)


  input_ids2 = encoded_data2['input_ids']
  attention_masks2 = encoded_data2['attention_mask']

  dataset2 = TensorDataset(input_ids2, attention_masks2, targets2, stances2)


  batch_size2 = 8

  dataloader2 = DataLoader(dataset2, 
                                sampler=RandomSampler(dataset2), 
                                batch_size=batch_size2)

In [ ]:

# I suggest 0.75 for threshold
print('Enter bleu score threshold value : ')
threshold = 0.75
model.eval()
data = pd.DataFrame(columns=df.columns)
for batch in tqdm(dataloader):
    targets = batch[2]
    stances = batch[3]
    queries = tokenizer.batch_decode(batch[0])
    batch = tuple(b.to('cuda') for b in batch)
    with torch.no_grad():  
        outputs = tokenizer.batch_decode(model.generate(
            batch[0],
            attention_mask=batch[1],
            max_length=280,
            do_sample=True,
            top_k=120,
            top_p=0.95,
            early_stopping=True,
            num_return_sequences=1
            ), skip_special_tokens=True,clean_up_tokenization_spaces=True)
        for query, output, target, stance in zip(queries, outputs, targets, stances):
            o = str(output).strip('<pad>').strip('</s>')
            if get_bleu_score(query, o) > threshold:
            # can verify bleu score here 
                data.loc[len(data.index)] = [o, int(target), int(stance)] # adding the new data to the dataframe
#data.to_csv(f'/content/drive/MyDrive/CSE 573: Stance Detection /Datasets/SemEval-2016/modified/train_data_paraphrased.csv', index=False)
#print('Augmented data file stored')

Enter bleu score threshold value : 


  0%|          | 0/176 [00:00<?, ?it/s]

In [ ]:

if usingProcon:
  # I suggest 0.75 for threshold
  print('Enter bleu score threshold value : ')
  threshold = 0.75
  model.eval()
  data2 = pd.DataFrame(columns=df2.columns)
  for batch in tqdm(dataloader2):
      targets = batch[2]
      stances = batch[3]
      queries = tokenizer.batch_decode(batch[0])
      batch = tuple(b.to('cuda') for b in batch)
      with torch.no_grad():  
          outputs = tokenizer.batch_decode(model.generate(
              batch[0],
              attention_mask=batch[1],
              max_length=280,
              do_sample=True,
              top_k=120,
              top_p=0.95,
              early_stopping=True,
              num_return_sequences=1
              ), skip_special_tokens=True,clean_up_tokenization_spaces=True)
          for query, output, target, stance in zip(queries, outputs, targets, stances):
              o = str(output).strip('<pad>').strip('</s>')
              if get_bleu_score(query, o) > threshold:
              # can verify bleu score here 
                  data2.loc[len(data2.index)] = [o, int(target), int(stance)] # adding the new data to the dataframe
  #data.to_csv(f'/content/drive/MyDrive/CSE 573: Stance Detection /Datasets/SemEval-2016/modified/train_data_paraphrased.csv', index=False)
  #print('Augmented data file stored')

Enter bleu score threshold value : 


  0%|          | 0/748 [00:00<?, ?it/s]

In [ ]:
# replacing target ids with target values
target_dict = {v : k for k, v in target_dict.items()}
data['Target'] = data.Target.replace(target_dict)
if usingProcon:
  target_dict2 = {v : k for k, v in target_dict2.items()}
  data2['Target'] = data2.Target.replace(target_dict2)

In [ ]:
#Combine Procon and Semeval!
if usingProcon:
  final_paraphrased = pd.concat([data,data2], ignore_index=True)
  final_paraphrased.to_csv(f'/content/drive/MyDrive/CSE 573: Stance Detection /Datasets/Both/paraphrased.csv')
else: 
  final_paraphrased = data.copy()
  final_paraphrased.to_csv(f'/content/drive/MyDrive/CSE 573: Stance Detection /Datasets/SemEval-2016/modified/paraphrased.csv')